In [1]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, classification_report
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Data Preparation & Exploration

In [3]:
testData = pd.read_csv('test.csv')
trainData = pd.read_csv('train.csv')
y = trainData.label
X = trainData.drop('label', axis=1)
global modelNum
modelNum = 1

In [45]:
print(X.isnull().any()==True)

pixel0      False
pixel1      False
pixel2      False
pixel3      False
pixel4      False
            ...  
pixel779    False
pixel780    False
pixel781    False
pixel782    False
pixel783    False
Length: 784, dtype: bool


In [46]:
X

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
y

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [48]:
print(X.shape)
print(y.shape)

(42000, 784)
(42000,)


In [4]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state = 42)

In [6]:
y_train

26906    0
38602    9
10316    2
27806    3
31367    2
        ..
11345    9
11769    6
28685    5
8159     8
11703    1
Name: label, Length: 22050, dtype: int64

In [7]:
y_train = keras.utils.to_categorical(y_train, 10)
y_valid = keras.utils.to_categorical(y_valid, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [8]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

# Creating Neural Network Models

In [9]:
twoLayer10NodeModel = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(784,)),
    keras.layers.Dense(10, activation = 'relu'),
    keras.layers.Dense(10, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

In [10]:
def fitModel(model, loss, optimizer):
    global history
    model.compile(loss = loss, optimizer = optimizer, metrics = ['accuracy'])
    history = model.fit(X_train, y_train, epochs = 30, validation_data = (X_valid, y_valid))

def predictModel(model):
    global trainDF
    global modelNum
    print('Test accuracy',model.evaluate(X_test,y_test)[1])
    y_pred = model.predict(testData)
    y_pred_classes = np.argmax(y_pred, axis = 1)
    trainDF = pd.DataFrame(y_pred_classes, columns = ['Label'])
    trainDF.index.name = 'ImageId'
    trainDF.index += 1
    trainDF.to_csv(f'model{modelNum}prediction.csv')
    modelNum+=1

## 2 Layers, 10 Nodes

In [11]:
start = dt.datetime.now()
fitModel(twoLayer10NodeModel,'categorical_crossentropy','adam')
end = dt.datetime.now()
twoLayer10NodeModelFitTime = end - start
print(f'Time taken to fit 2 layers with 10 nodes each: {twoLayer10NodeModelFitTime}')
# Kaggle Score: 0.82828, Username: Deemal Patel

Epoch 1/30
690/690 [==============================] - 3s 3ms/step - loss: 6.1829 - accuracy: 0.1696 - val_loss: 1.7055 - val_accuracy: 0.3697
Epoch 2/30
690/690 [==============================] - 1s 1ms/step - loss: 1.6202 - accuracy: 0.3751 - val_loss: 1.4689 - val_accuracy: 0.4061
Epoch 3/30
690/690 [==============================] - 1s 1ms/step - loss: 1.4369 - accuracy: 0.4163 - val_loss: 1.3592 - val_accuracy: 0.4653
Epoch 4/30
690/690 [==============================] - 1s 1ms/step - loss: 1.3200 - accuracy: 0.4527 - val_loss: 1.3021 - val_accuracy: 0.4592
Epoch 5/30
690/690 [==============================] - 1s 1ms/step - loss: 1.2626 - accuracy: 0.4690 - val_loss: 1.2308 - val_accuracy: 0.5038
Epoch 6/30
690/690 [==============================] - 1s 1ms/step - loss: 1.1862 - accuracy: 0.5150 - val_loss: 1.1615 - val_accuracy: 0.5597
Epoch 7/30
690/690 [==============================] - 1s 1ms/step - loss: 1.1348 - accuracy: 0.5427 - val_loss: 1.0968 - val_accuracy: 0.5556
Epoch 

In [12]:
predictModel(twoLayer10NodeModel)

394/394 [==============================] - 0s 1ms/step - loss: 0.6178 - accuracy: 0.8267
Test accuracy 0.8266666531562805


In [13]:
start = dt.datetime.now()
fitModel(twoLayer10NodeModel,'categorical_crossentropy','sgd')
end = dt.datetime.now()
twoLayer10NodeModelFitTime = end - start
print(f'Time taken to fit 2 layers with 10 nodes each: {twoLayer10NodeModelFitTime}')

Epoch 1/30
690/690 [==============================] - 2s 2ms/step - loss: 2.6486 - accuracy: 0.3008 - val_loss: 1.5713 - val_accuracy: 0.4106
Epoch 2/30
690/690 [==============================] - 1s 2ms/step - loss: 1.6033 - accuracy: 0.3758 - val_loss: 1.5078 - val_accuracy: 0.3780
Epoch 3/30
690/690 [==============================] - 1s 1ms/step - loss: 1.5391 - accuracy: 0.3734 - val_loss: 1.6506 - val_accuracy: 0.2973
Epoch 4/30
690/690 [==============================] - 1s 1ms/step - loss: 1.6566 - accuracy: 0.3022 - val_loss: 1.6134 - val_accuracy: 0.3091
Epoch 5/30
690/690 [==============================] - 1s 1ms/step - loss: 1.5971 - accuracy: 0.3100 - val_loss: 1.5891 - val_accuracy: 0.3056
Epoch 6/30
690/690 [==============================] - 1s 1ms/step - loss: 1.5779 - accuracy: 0.3103 - val_loss: 1.5828 - val_accuracy: 0.3063
Epoch 7/30
690/690 [==============================] - 1s 1ms/step - loss: 1.5842 - accuracy: 0.3039 - val_loss: 1.6956 - val_accuracy: 0.2819
Epoch 

## 2 Layers, 20 Nodes

In [14]:
twoLayer20NodeModel = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(784,)),
    keras.layers.Dense(20, activation = 'relu'),
    keras.layers.Dense(20, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

In [15]:
start = dt.datetime.now()
fitModel(twoLayer20NodeModel,'categorical_crossentropy','adam')
end = dt.datetime.now()
twoLayer20NodeModelFitTime = end - start
print(f'Time taken to fit 2 layers with 20 nodes each: {twoLayer20NodeModelFitTime}')
# Kaggle Score: 0.79800, Username: Deemal Patel

Epoch 1/30
690/690 [==============================] - 1s 1ms/step - loss: 8.1893 - accuracy: 0.2541 - val_loss: 1.5790 - val_accuracy: 0.4339
Epoch 2/30
690/690 [==============================] - 1s 1ms/step - loss: 1.5183 - accuracy: 0.4333 - val_loss: 1.2922 - val_accuracy: 0.4831
Epoch 3/30
690/690 [==============================] - 1s 1ms/step - loss: 1.2821 - accuracy: 0.4938 - val_loss: 1.2162 - val_accuracy: 0.5127
Epoch 4/30
690/690 [==============================] - 1s 1ms/step - loss: 1.1791 - accuracy: 0.5136 - val_loss: 1.0602 - val_accuracy: 0.5924
Epoch 5/30
690/690 [==============================] - 1s 1ms/step - loss: 1.0083 - accuracy: 0.6024 - val_loss: 1.0313 - val_accuracy: 0.6004
Epoch 6/30
690/690 [==============================] - 1s 1ms/step - loss: 0.9469 - accuracy: 0.6253 - val_loss: 0.9058 - val_accuracy: 0.6773
Epoch 7/30
690/690 [==============================] - 1s 1ms/step - loss: 0.8403 - accuracy: 0.6946 - val_loss: 0.8384 - val_accuracy: 0.6935
Epoch 

In [16]:
predictModel(twoLayer20NodeModel)

394/394 [==============================] - 0s 731us/step - loss: 0.6541 - accuracy: 0.7987
Test accuracy 0.7986508011817932


In [17]:
start = dt.datetime.now()
fitModel(twoLayer20NodeModel,'categorical_crossentropy','sgd')
end = dt.datetime.now()
twoLayer20NodeModelFitTime = end - start
print(f'Time taken to fit 2 layers with 20 nodes each: {twoLayer20NodeModelFitTime}')

Epoch 1/30
690/690 [==============================] - 1s 2ms/step - loss: 1.8952 - accuracy: 0.4240 - val_loss: 1.2289 - val_accuracy: 0.5358
Epoch 2/30
690/690 [==============================] - 1s 1ms/step - loss: 1.0839 - accuracy: 0.5516 - val_loss: 1.2587 - val_accuracy: 0.4637
Epoch 3/30
690/690 [==============================] - 1s 1ms/step - loss: 0.9561 - accuracy: 0.5857 - val_loss: 0.9137 - val_accuracy: 0.6299
Epoch 4/30
690/690 [==============================] - 1s 1ms/step - loss: 1.1032 - accuracy: 0.5600 - val_loss: 0.9186 - val_accuracy: 0.6341
Epoch 5/30
690/690 [==============================] - 1s 1ms/step - loss: 0.9521 - accuracy: 0.6093 - val_loss: 1.2589 - val_accuracy: 0.4989
Epoch 6/30
690/690 [==============================] - 1s 962us/step - loss: 1.2387 - accuracy: 0.4890 - val_loss: 1.3368 - val_accuracy: 0.4758
Epoch 7/30
690/690 [==============================] - 1s 1ms/step - loss: 1.1310 - accuracy: 0.5332 - val_loss: 1.0209 - val_accuracy: 0.5642
Epoc

## 5 Layers, 10 Nodes

In [18]:
fiveLayer10NodeModel = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(784,)),
    keras.layers.Dense(10, activation = 'relu'),
    keras.layers.Dense(10, activation = 'relu'),
    keras.layers.Dense(10, activation = 'relu'),
    keras.layers.Dense(10, activation = 'relu'),
    keras.layers.Dense(10, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

In [19]:
start = dt.datetime.now()
fitModel(fiveLayer10NodeModel,'categorical_crossentropy','adam')
end = dt.datetime.now()
fiveLayer10NodeModelFitTime = end - start
print(f'Time taken to fit 5 layers with 10 nodes each: {fiveLayer10NodeModelFitTime}')
# Kaggle Score: 0.89285, User:Deemal Patel

Epoch 1/30
690/690 [==============================] - 1s 1ms/step - loss: 2.2024 - accuracy: 0.2985 - val_loss: 1.1073 - val_accuracy: 0.6189
Epoch 2/30
690/690 [==============================] - 1s 1ms/step - loss: 0.9475 - accuracy: 0.6586 - val_loss: 0.8694 - val_accuracy: 0.7086
Epoch 3/30
690/690 [==============================] - 1s 1ms/step - loss: 0.7708 - accuracy: 0.7208 - val_loss: 0.7611 - val_accuracy: 0.7200
Epoch 4/30
690/690 [==============================] - 1s 1ms/step - loss: 0.7175 - accuracy: 0.7351 - val_loss: 0.7053 - val_accuracy: 0.7314
Epoch 5/30
690/690 [==============================] - 1s 1ms/step - loss: 0.6881 - accuracy: 0.7376 - val_loss: 0.7083 - val_accuracy: 0.7536
Epoch 6/30
690/690 [==============================] - 1s 1ms/step - loss: 0.6577 - accuracy: 0.7593 - val_loss: 0.6586 - val_accuracy: 0.7776
Epoch 7/30
690/690 [==============================] - 1s 1ms/step - loss: 0.5998 - accuracy: 0.7952 - val_loss: 0.5949 - val_accuracy: 0.8114
Epoch 

In [20]:
predictModel(fiveLayer10NodeModel)

394/394 [==============================] - 0s 704us/step - loss: 0.3804 - accuracy: 0.8937
Test accuracy 0.8937301635742188


## 5 Layers, 20 Nodes

In [21]:
fiveLayer20NodeModel = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(784,)),
    keras.layers.Dense(20, activation = 'relu'),
    keras.layers.Dense(20, activation = 'relu'),
    keras.layers.Dense(20, activation = 'relu'),
    keras.layers.Dense(20, activation = 'relu'),
    keras.layers.Dense(20, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

In [22]:
start = dt.datetime.now()
fitModel(fiveLayer20NodeModel,'categorical_crossentropy','adam')
end = dt.datetime.now()
fiveLayer20NodeModelFitTime = end - start
print(f'Time taken to fit 5 layers with 20 nodes each: {fiveLayer20NodeModelFitTime}')
# Kaggle Score: 0.91928, User: Deemal Patel

Epoch 1/30
690/690 [==============================] - 1s 1ms/step - loss: 3.0041 - accuracy: 0.2936 - val_loss: 0.8736 - val_accuracy: 0.6967
Epoch 2/30
690/690 [==============================] - 1s 1ms/step - loss: 0.8004 - accuracy: 0.7367 - val_loss: 0.6332 - val_accuracy: 0.7940
Epoch 3/30
690/690 [==============================] - 1s 1ms/step - loss: 0.6021 - accuracy: 0.8099 - val_loss: 0.5461 - val_accuracy: 0.8336
Epoch 4/30
690/690 [==============================] - 1s 1ms/step - loss: 0.4943 - accuracy: 0.8480 - val_loss: 0.5029 - val_accuracy: 0.8483
Epoch 5/30
690/690 [==============================] - 1s 1ms/step - loss: 0.4496 - accuracy: 0.8630 - val_loss: 0.4617 - val_accuracy: 0.8707
Epoch 6/30
690/690 [==============================] - 1s 1ms/step - loss: 0.3972 - accuracy: 0.8837 - val_loss: 0.4120 - val_accuracy: 0.8766
Epoch 7/30
690/690 [==============================] - 1s 1ms/step - loss: 0.3586 - accuracy: 0.8938 - val_loss: 0.3842 - val_accuracy: 0.8879
Epoch 

In [23]:
predictModel(fiveLayer20NodeModel)

394/394 [==============================] - 0s 642us/step - loss: 0.3156 - accuracy: 0.9184
Test accuracy 0.9184126853942871


## 5 Layer, 50 Node

In [24]:
fiveLayer50NodeModel = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(784,)),
    keras.layers.Dense(50, activation = 'relu'),
    keras.layers.Dense(50, activation = 'relu'),
    keras.layers.Dense(50, activation = 'relu'),
    keras.layers.Dense(50, activation = 'relu'),
    keras.layers.Dense(50, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

In [25]:
start = dt.datetime.now()
fitModel(fiveLayer50NodeModel,'categorical_crossentropy','adam')
end = dt.datetime.now()
fiveLayer50NodeModelFitTime = end - start
print(f'Time taken to fit 5 layers with 50 nodes each: {fiveLayer50NodeModelFitTime}')
# Kaggle Score: 0.95139, Deemal Patel

Epoch 1/30
690/690 [==============================] - 2s 2ms/step - loss: 2.1409 - accuracy: 0.5920 - val_loss: 0.4207 - val_accuracy: 0.8771
Epoch 2/30
690/690 [==============================] - 1s 1ms/step - loss: 0.3880 - accuracy: 0.8864 - val_loss: 0.3498 - val_accuracy: 0.9001
Epoch 3/30
690/690 [==============================] - 1s 1ms/step - loss: 0.2616 - accuracy: 0.9213 - val_loss: 0.2750 - val_accuracy: 0.9203
Epoch 4/30
690/690 [==============================] - 1s 1ms/step - loss: 0.2091 - accuracy: 0.9375 - val_loss: 0.2467 - val_accuracy: 0.9298
Epoch 5/30
690/690 [==============================] - 1s 1ms/step - loss: 0.1884 - accuracy: 0.9430 - val_loss: 0.2520 - val_accuracy: 0.9294
Epoch 6/30
690/690 [==============================] - 1s 1ms/step - loss: 0.1503 - accuracy: 0.9537 - val_loss: 0.2318 - val_accuracy: 0.9390
Epoch 7/30
690/690 [==============================] - 1s 1ms/step - loss: 0.1446 - accuracy: 0.9555 - val_loss: 0.2378 - val_accuracy: 0.9404
Epoch 

In [26]:
predictModel(fiveLayer50NodeModel)

394/394 [==============================] - 0s 700us/step - loss: 0.2472 - accuracy: 0.9545
Test accuracy 0.9545238018035889


## 5 Layer, 100 Node

In [27]:
fiveLayer100NodeModel = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(784,)),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

In [28]:
start = dt.datetime.now()
fitModel(fiveLayer100NodeModel,'categorical_crossentropy','adam')
end = dt.datetime.now()
fiveLayer100NodeModelFitTime = end - start
print(f'Time taken to fit 5 layers with 100 nodes each: {fiveLayer100NodeModelFitTime}')
# Kaggle Score: 0.96514, Deemal Patel

Epoch 1/30
690/690 [==============================] - 2s 2ms/step - loss: 1.9150 - accuracy: 0.6888 - val_loss: 0.4015 - val_accuracy: 0.8888
Epoch 2/30
690/690 [==============================] - 1s 2ms/step - loss: 0.2984 - accuracy: 0.9127 - val_loss: 0.2458 - val_accuracy: 0.9297
Epoch 3/30
690/690 [==============================] - 1s 2ms/step - loss: 0.1886 - accuracy: 0.9462 - val_loss: 0.2282 - val_accuracy: 0.9350
Epoch 4/30
690/690 [==============================] - 1s 2ms/step - loss: 0.1611 - accuracy: 0.9504 - val_loss: 0.2258 - val_accuracy: 0.9369
Epoch 5/30
690/690 [==============================] - 1s 2ms/step - loss: 0.1380 - accuracy: 0.9574 - val_loss: 0.2100 - val_accuracy: 0.9445
Epoch 6/30
690/690 [==============================] - 1s 2ms/step - loss: 0.1221 - accuracy: 0.9611 - val_loss: 0.3055 - val_accuracy: 0.9260
Epoch 7/30
690/690 [==============================] - 1s 2ms/step - loss: 0.1282 - accuracy: 0.9619 - val_loss: 0.2146 - val_accuracy: 0.9431
Epoch 

In [29]:
predictModel(fiveLayer100NodeModel)

394/394 [==============================] - 0s 793us/step - loss: 0.1875 - accuracy: 0.9675
Test accuracy 0.9674603343009949


## 5 Layer, 150 Node

In [30]:
fiveLayer150NodeModel = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(784,)),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

In [31]:
start = dt.datetime.now()
fitModel(fiveLayer150NodeModel,'categorical_crossentropy','adam')
end = dt.datetime.now()
fiveLayer150NodeModelFitTime = end - start
print(f'Time taken to fit 5 layers with 150 nodes each: {fiveLayer150NodeModelFitTime}')
# Kaggle Score 0.95967,  Deemal Patel

Epoch 1/30
690/690 [==============================] - 2s 2ms/step - loss: 2.3259 - accuracy: 0.7246 - val_loss: 0.3182 - val_accuracy: 0.9086
Epoch 2/30
690/690 [==============================] - 1s 2ms/step - loss: 0.2542 - accuracy: 0.9258 - val_loss: 0.2412 - val_accuracy: 0.9333
Epoch 3/30
690/690 [==============================] - 1s 2ms/step - loss: 0.1853 - accuracy: 0.9464 - val_loss: 0.2290 - val_accuracy: 0.9381
Epoch 4/30
690/690 [==============================] - 1s 2ms/step - loss: 0.1750 - accuracy: 0.9494 - val_loss: 0.2290 - val_accuracy: 0.9359
Epoch 5/30
690/690 [==============================] - 1s 2ms/step - loss: 0.1426 - accuracy: 0.9581 - val_loss: 0.1864 - val_accuracy: 0.9459
Epoch 6/30
690/690 [==============================] - 1s 2ms/step - loss: 0.1152 - accuracy: 0.9653 - val_loss: 0.2115 - val_accuracy: 0.9412
Epoch 7/30
690/690 [==============================] - 1s 2ms/step - loss: 0.1019 - accuracy: 0.9688 - val_loss: 0.2182 - val_accuracy: 0.9456
Epoch 

In [32]:
predictModel(fiveLayer150NodeModel)

394/394 [==============================] - 0s 951us/step - loss: 0.2804 - accuracy: 0.9629
Test accuracy 0.962936520576477


## 10 Layer, 150 Node

In [33]:
tenLayer150NodeModel = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(784,)),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(150, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

In [34]:
start = dt.datetime.now()
fitModel(tenLayer150NodeModel,'categorical_crossentropy','adam')
end = dt.datetime.now()
tenLayer150NodeModelFitTime = end - start
print(f'Time taken to fit 10 layers with 150 nodes each: {tenLayer150NodeModelFitTime}')
# Kaggle Score 0.95528

Epoch 1/30
690/690 [==============================] - 2s 3ms/step - loss: 0.8543 - accuracy: 0.7384 - val_loss: 0.2518 - val_accuracy: 0.9307
Epoch 2/30
690/690 [==============================] - 2s 2ms/step - loss: 0.2363 - accuracy: 0.9357 - val_loss: 0.2761 - val_accuracy: 0.9268
Epoch 3/30
690/690 [==============================] - 2s 2ms/step - loss: 0.1808 - accuracy: 0.9506 - val_loss: 0.2252 - val_accuracy: 0.9418
Epoch 4/30
690/690 [==============================] - 2s 2ms/step - loss: 0.1480 - accuracy: 0.9599 - val_loss: 0.1962 - val_accuracy: 0.9531
Epoch 5/30
690/690 [==============================] - 2s 2ms/step - loss: 0.1285 - accuracy: 0.9665 - val_loss: 0.1746 - val_accuracy: 0.9576
Epoch 6/30
690/690 [==============================] - 2s 2ms/step - loss: 0.1178 - accuracy: 0.9708 - val_loss: 0.1846 - val_accuracy: 0.9567
Epoch 7/30
690/690 [==============================] - 2s 2ms/step - loss: 0.1174 - accuracy: 0.9724 - val_loss: 0.2025 - val_accuracy: 0.9562
Epoch 

In [35]:
predictModel(tenLayer150NodeModel)

394/394 [==============================] - 0s 1ms/step - loss: 0.2433 - accuracy: 0.9593
Test accuracy 0.9592857360839844


## 10 Layer, 200 Node

In [36]:
tenLayer200NodeModel = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(784,)),
    keras.layers.Dense(200, activation = 'relu'),
    keras.layers.Dense(200, activation = 'relu'),
    keras.layers.Dense(200, activation = 'relu'),
    keras.layers.Dense(200, activation = 'relu'),
    keras.layers.Dense(200, activation = 'relu'),
    keras.layers.Dense(200, activation = 'relu'),
    keras.layers.Dense(200, activation = 'relu'),
    keras.layers.Dense(200, activation = 'relu'),
    keras.layers.Dense(200, activation = 'relu'),
    keras.layers.Dense(200, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

In [37]:
start = dt.datetime.now()
fitModel(tenLayer200NodeModel,'categorical_crossentropy','adam')
end = dt.datetime.now()
tenLayer200NodeModelFitTime = end - start
print(f'Time taken to fit 10 layers with 150 nodes each: {tenLayer200NodeModelFitTime}')
# Kaggle Score: 0.96203

Epoch 1/30
690/690 [==============================] - 3s 4ms/step - loss: 0.9061 - accuracy: 0.7468 - val_loss: 0.2373 - val_accuracy: 0.9340
Epoch 2/30
690/690 [==============================] - 2s 3ms/step - loss: 0.2415 - accuracy: 0.9381 - val_loss: 0.3030 - val_accuracy: 0.9190
Epoch 3/30
690/690 [==============================] - 2s 3ms/step - loss: 0.2702 - accuracy: 0.9274 - val_loss: 0.2032 - val_accuracy: 0.9478
Epoch 4/30
690/690 [==============================] - 2s 3ms/step - loss: 0.1666 - accuracy: 0.9606 - val_loss: 0.2088 - val_accuracy: 0.9503
Epoch 5/30
690/690 [==============================] - 2s 3ms/step - loss: 0.1177 - accuracy: 0.9708 - val_loss: 0.2782 - val_accuracy: 0.9465
Epoch 6/30
690/690 [==============================] - 2s 3ms/step - loss: 0.1446 - accuracy: 0.9686 - val_loss: 0.2196 - val_accuracy: 0.9518
Epoch 7/30
690/690 [==============================] - 2s 3ms/step - loss: 0.1198 - accuracy: 0.9711 - val_loss: 0.2120 - val_accuracy: 0.9603
Epoch 

In [38]:
predictModel(tenLayer200NodeModel)

394/394 [==============================] - 1s 1ms/step - loss: 0.3202 - accuracy: 0.9640
Test accuracy 0.9639682769775391


In [39]:
nodesPerModel = [10,20,10,20,50,100,150,150,200]
layersPerModel = [2,2,5,5,5,5,5,10,10]
timePerModel = [twoLayer10NodeModelFitTime,twoLayer20NodeModelFitTime,fiveLayer10NodeModelFitTime,
               fiveLayer20NodeModelFitTime,fiveLayer50NodeModelFitTime,fiveLayer100NodeModelFitTime,
               fiveLayer150NodeModelFitTime,tenLayer150NodeModelFitTime,tenLayer200NodeModelFitTime]
trainingSetAccuracy = [0.8483, 0.8739, 0.9228, 0.9506, 0.9913, 0.9921, 0.9943,0.9928, 0.9909]
testingSetAccuracy = [0.826, 0.7987, 0.8937, 0.9184, 0.9545, 0.9675, 0.9629,0.9593, 0.9640]

In [43]:
summaryDF = pd.DataFrame({
    'Number of Layers': layersPerModel,
    'Nodes Per Layer': nodesPerModel,
    'Processing Time':timePerModel,
    'Traing Set Accuracy':trainingSetAccuracy,
    'Testing Set Accuracy':testingSetAccuracy
})
summaryDF

,Number of Layers,Nodes Per Layer,Processing Time,Traing Set Accuracy,Testing Set Accuracy
0,2,10,00:00:24.243876,0.8483,0.8260
1,2,20,00:00:22.404917,0.8739,0.7987
2,5,10,00:00:25.039713,0.9228,0.8937
3,5,20,00:00:23.558120,0.9506,0.9184
4,5,50,00:00:28.519816,0.9913,0.9545
5,5,100,00:00:34.425971,0.9921,0.9675
6,5,150,00:00:41.499968,0.9943,0.9629
7,10,150,00:00:50.160994,0.9928,0.9593
8,10,200,00:01:05.512456,0.9909,0.9640


# Management Questions

1. We begin by exploring the data within this dataset. We can see that that the training data set contains 42000 rows of data and 784 columns of explanatory variables. Each column represent a single pixel of the image and each image is a 28 x 28 pixel. We look at the corresponding labels to get an idea of how which number the explanatory variables the image belongs to. Because these labels will be used within a neural network for the purpose of properly determining the image, we need to convert the labels in to categoricals

2. We develop multiple neural network models with different hyperparameters to tune the model in order to get a high accuracy not only from training but also when we test it. The modeling method being used here is a MLP, Multiple Layer Perceptron. In order to properly implement the MLP are using the Sequential API. The Sequential API will help us with our classification MLP to properly determine the digit from the test data. We create our train, test, and validation set before we start the implemention. We begin implementing the neural network by creating a keras model which comes with tensorflow which is the package that will allow us to build our model. The model requires and input layer, the hidden layer, and the outer layer. We must transform the model to be ready for a 784 input shape because our dataset has 784 expalantory variables that help produce the labels. The layers use the ReLU, or Rectified Linear Unit, which is a good choice to use when starting to create the models. We use a softmax in the outer layer because it calculates the probability that the data belongs to a specific class. The sum of all the softmax classification probabilities should add up to one. Once the layers and have been set up we want to compile the model using the categorical_crossentropy as our loss function. We want to optimize the model with an adam optimizer which is an adaptive learning rate algorithm. The metric used is accuracy because we want to see how accurate the model is when correctly classifying the images.

3. We have created 9 different models with a different mix of layers and nodes per layer to determine the accuracy of each one. The accuracy will tell us how well the model performed with the training set and the validation set. If the validation set's accuracy is fairly close to the training set's and the accuracy is a score that we like to see, for example 95%, then it is fair to say that there is little to no overfitting/underfitting occuring and we have a solid model. Analyzing at what we have created are two, two layer models each where the first one has 10 nodes per layer and the other one has 20 nodes per layer. The accuracy scores we get for the training and testing set are not bad coming to around 84.8% for the first model and 87.3% for the second model. This can definitely be improved so we have 5 more models that have 5 layers each with nodes that are 10,20,50,100,150, respectively. The accuracy came out a lot better with the 5 layers getting 92.2%, 95%, 99.1%, 99.2%, 99.4%. The testing set came very close to around 96% for each but this is to be expected as the model would favor the training set a little more. We also ran 2 10 layer models with 150 nodes and 200 nodes. These also came to about 99%. At this point because the accuracies are so close to one another we want to look at the processing time. The best accuracy to time ratio would probably be model 6 because it has a short processing time with a high accuracy.

4. From the benchmark dataframe we can see that going any higher on the layers would not provide much benefit. Because the processing time increase by 10 seconds for every 50 nodes added it would be a waste of time to perform any more models only to maybe see an improvement of 0.1% improvement in the model. For our use the instituitons use, I would recommend we use the 5 layer model with 100 nodes per layer because of the good accuracy to processing time ratio. We get an incredibly good accuracy score while not having to spend more time determining how many more models to test. The settings for this model used the Sequential API with a categorical_crossentropy loss function and the adam optimizer in order to achieve the best results. 